In [339]:
import pandas as pd
import numpy

# data
restaurants = pd.read_csv("https://raw.githubusercontent.com/xoyeon/PDPTW/main/grubhub/0o100t100s1p100/restaurants.txt", sep = '\t')
orders = pd.read_csv("https://raw.githubusercontent.com/xoyeon/PDPTW/main/grubhub/0o100t100s1p100/orders.txt", sep = '\t')
couriers = pd.read_csv("https://raw.githubusercontent.com/xoyeon/PDPTW/main/grubhub/0o100t100s1p100/couriers.txt", sep = '\t')
rest_orders = pd.merge(restaurants, orders, how='inner', on=['restaurant'])
rest_orders.columns = ['restaurant', 'rx', 'ry', 'order', 'ox', 'oy', 'placement_time', 'ready_time']

# 주문 건
# order = rest_orders.sample(n=1)
order = ['r1',8708,5633,'o1',9131,7497,743,753]
print("order", order)
# 근무중인 라이더
rider = couriers[(couriers['on_time'] <= order[6]) & (couriers['off_time'] >= order[7])]
working_rider = len(rider)
print("근무중인 라이더", working_rider)
"""
Genetic algorithm parameters:
    Mating pool size
    Population size
"""


# Defining the population size.
sol_per_pop = 5
num_parents_mating = 4

# Inputs of the equation. # 내가 업데이트 해야하는 값
dist_x = numpy.array(rider['x'])
dist_y = numpy.array(rider['y'])
on_time = numpy.array(rider['on_time'])
off_time = numpy.array(rider['off_time'])

equation_inputs = numpy.stack((dist_x, dist_y, on_time, off_time), axis=1)
print("equation_inputs",equation_inputs)

# Number of the weights we are looking to optimize.
num_weights = len(equation_inputs)
print("num_weights",num_weights)

# The population will have sol_per_pop chromosome where each chromosome has num_weights genes.
pop_size = (sol_per_pop,num_weights)

#Creating the initial population.
delivered_num = numpy.random.uniform(low=3, high=10, size=pop_size).astype(int)
speed = numpy.random.uniform(low=30, high=60, size=pop_size).astype(int)
trustiness = numpy.full(pop_size, 0.90)

new_population = numpy.stack((delivered_num, speed, trustiness), axis=1)
print("new_population",new_population)

order ['r1', 8708, 5633, 'o1', 9131, 7497, 743, 753]
근무중인 라이더 9
equation_inputs [[ 3810  7415   690   810]
 [ 3217  9891   690   840]
 [11678  3104   690   810]
 [ 7651  4332   690   810]
 [10855  7784   690   810]
 [ 7038  5694   690   840]
 [ 5013  2778   690   810]
 [ 8236  1518   690   810]
 [ 4981  4827   690   840]]
num_weights 9
new_population [[[ 8.   6.   3.   3.   7.   8.   4.   3.   8. ]
  [55.  58.  45.  32.  32.  47.  33.  41.  50. ]
  [ 0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9]]

 [[ 7.   5.   9.   5.   3.   3.   9.   3.   8. ]
  [30.  51.  32.  30.  35.  56.  37.  36.  57. ]
  [ 0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9]]

 [[ 6.   8.   7.   5.   9.   6.   6.   9.   7. ]
  [47.  32.  59.  46.  32.  48.  56.  58.  53. ]
  [ 0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9]]

 [[ 8.   3.   6.   3.   8.   3.   9.   6.   5. ]
  [56.  43.  46.  40.  41.  44.  40.  59.  31. ]
  [ 0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9]]

 [[ 8.   4.   9.   8.   8.   3.   6.   4.   3. ]
  

In [530]:
def standard_deviation(equation_inputs, pop):
    std = []
    for s in range(len(pop)):
        mean = sum(pop[:,0][s]) / len(equation_inputs)
        var = ((sum(pop[:,0][s]) - mean)**2) / len(equation_inputs)
        std.append(var ** 0.5)
    std = numpy.array(std)
    return std
std = standard_deviation(equation_inputs, new_population)
print(std.shape)
print(std)
print(type(std))

(5,)
[14.81481481 15.40740741 18.66666667 15.11111111 15.7037037 ]
<class 'numpy.ndarray'>


In [563]:
def cal_pop_fitness(equation_inputs, order, pop):
    std = standard_deviation(equation_inputs, pop)
    dist = calculate_distance(order, equation_inputs)
    time = calculate_time(order, equation_inputs, pop)
    # Calculating the fitness value of each solution in the current population.
    # The fitness function calulates the sum of products between each input and its corresponding weight.
    fitness = []
    for w in range(len(new_population)):
        dpt = distance / time
        fitness.append(1/std[w]+dt[w])
    return fitness

fitness = cal_pop_fitness(equation_inputs, order, new_population)

In [564]:
fitness

[array([55.0675, 58.0675, 45.0675, 32.0675, 32.0675, 47.0675, 33.0675,
        41.0675, 50.0675]),
 array([30.06490385, 51.06490385, 32.06490385, 30.06490385, 35.06490385,
        56.06490385, 37.06490385, 36.06490385, 57.06490385]),
 array([47.05357143, 32.05357143, 59.05357143, 46.05357143, 32.05357143,
        48.05357143, 56.05357143, 58.05357143, 53.05357143]),
 array([56.06617647, 43.06617647, 46.06617647, 40.06617647, 41.06617647,
        44.06617647, 40.06617647, 59.06617647, 31.06617647]),
 array([46.06367925, 54.06367925, 51.06367925, 53.06367925, 32.06367925,
        50.06367925, 50.06367925, 59.06367925, 39.06367925])]

In [565]:
fitness[0]

array([55.0675, 58.0675, 45.0675, 32.0675, 32.0675, 47.0675, 33.0675,
       41.0675, 50.0675])

In [560]:
1/std[0] + dpt[0]

array([55.0675, 58.0675, 45.0675, 32.0675, 32.0675, 47.0675, 33.0675,
       41.0675, 50.0675])

In [547]:
distance/time

array([[55., 58., 45., 32., 32., 47., 33., 41., 50.],
       [30., 51., 32., 30., 35., 56., 37., 36., 57.],
       [47., 32., 59., 46., 32., 48., 56., 58., 53.],
       [56., 43., 46., 40., 41., 44., 40., 59., 31.],
       [46., 54., 51., 53., 32., 50., 50., 59., 39.]])

In [543]:
for w in range(len(new_population)):
    dt = distance / time
    print(1/std[w]+dt[w])

[55.0675 58.0675 45.0675 32.0675 32.0675 47.0675 33.0675 41.0675 50.0675]
[30.06490385 51.06490385 32.06490385 30.06490385 35.06490385 56.06490385
 37.06490385 36.06490385 57.06490385]
[47.05357143 32.05357143 59.05357143 46.05357143 32.05357143 48.05357143
 56.05357143 58.05357143 53.05357143]
[56.06617647 43.06617647 46.06617647 40.06617647 41.06617647 44.06617647
 40.06617647 59.06617647 31.06617647]
[46.06367925 54.06367925 51.06367925 53.06367925 32.06367925 50.06367925
 50.06367925 59.06367925 39.06367925]


In [536]:
dt[0]

array([55., 58., 45., 32., 32., 47., 33., 41., 50.])

In [534]:
dt = distance / time
dt

array([[55., 58., 45., 32., 32., 47., 33., 41., 50.],
       [30., 51., 32., 30., 35., 56., 37., 36., 57.],
       [47., 32., 59., 46., 32., 48., 56., 58., 53.],
       [56., 43., 46., 40., 41., 44., 40., 59., 31.],
       [46., 54., 51., 53., 32., 50., 50., 59., 39.]])

In [527]:
def calculate_distance(order, equation_inputs):
    distance = ((equation_inputs[:,0]-order[1])**2 + (equation_inputs[:,1]-order[2])**2 + (
        order[1]-order[4])**2 + (order[2]-order[5])**2) ** 0.5
    distance = numpy.round(distance, 2)
    return distance

distance = calculate_distance(order, equation_inputs)
print(distance.shape)
print(distance)
print(type(distance))

(9,)
[2657.4  4847.87 7701.84 3738.35 6851.52 3005.62 3348.8  5736.35 1960.58]
<class 'numpy.ndarray'>


In [528]:
def calculate_time(order, equation_inputs, pop):
    distance = calculate_distance(order, equation_inputs)
    time = distance / new_population[:,1]
    return time
time = calculate_time(order, equation_inputs, new_population)
print(time.shape)
print(time)

(5, 9)
[[ 48.31636364  83.58396552 171.152      116.8234375  214.11
   63.9493617  101.47878788 139.91097561  39.2116    ]
 [ 88.58        95.05627451 240.6825     124.61166667 195.75771429
   53.67178571  90.50810811 159.34305556  34.39614035]
 [ 56.54042553 151.4959375  130.53966102  81.26847826 214.11
   62.61708333  59.8         98.90258621  36.99207547]
 [ 47.45357143 112.74116279 167.43130435  93.45875    167.1102439
   68.30954545  83.72        97.22627119  63.24451613]
 [ 57.76956522  89.77537037 151.01647059  70.53490566 214.11
   60.1124      66.976       97.22627119  50.27128205]]


In [504]:
std + distance / time

ValueError: operands could not be broadcast together with shapes (9,1) (5,9) 

In [362]:
equation_inputs

array([[ 3810,  7415,   690,   810],
       [ 3217,  9891,   690,   840],
       [11678,  3104,   690,   810],
       [ 7651,  4332,   690,   810],
       [10855,  7784,   690,   810],
       [ 7038,  5694,   690,   840],
       [ 5013,  2778,   690,   810],
       [ 8236,  1518,   690,   810],
       [ 4981,  4827,   690,   840]], dtype=int64)

In [388]:
new_population[:,1]

array([[55., 58., 45., 32., 32., 47., 33., 41., 50.],
       [30., 51., 32., 30., 35., 56., 37., 36., 57.],
       [47., 32., 59., 46., 32., 48., 56., 58., 53.],
       [56., 43., 46., 40., 41., 44., 40., 59., 31.],
       [46., 54., 51., 53., 32., 50., 50., 59., 39.]])

In [390]:
order = ['r13',4607,5595,'o121',6363,5770,59,78]

In [409]:
new_population[:,2]

array([[0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
       [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
       [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
       [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
       [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9]])

In [421]:
pickup_dist = ((order[1]-equation_inputs[:,0])**2 + (order[2]-equation_inputs[:,1])**2)**0.5
pickup_time = pickup_dist / new_population[:,1] <= order[7]-order[6]
print(pickup_time == True)

[[False False False False False False False False  True]
 [False False False False False False False False  True]
 [False False False False False False False False  True]
 [False False False False False False False False False]
 [False False False False False False False False False]]


In [495]:
def calculate_trust(order, equation_inputs, pop):
    pickup_dist = ((order[1]-equation_inputs[:,0])**2 + (order[2]-equation_inputs[:,1])**2)**0.5
    pickup_time = pickup_dist / new_population[:,1] <= order[7]-order[6]
    return pickup_time
                  
trust = calculate_trust(order, equation_inputs, new_population)
trust

array([[False, False, False, False, False, False, False, False,  True],
       [False, False, False, False, False, False, False, False,  True],
       [False, False, False, False, False, False, False, False,  True],
       [False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False]])

In [497]:
if trust == False:
    print(new_population)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [333]:
distance/time[0]

array([41., 39., 50., 57., 44., 34., 46., 53., 54.])

In [334]:
std[0] + distance/time[0]

array([56.7037037, 54.7037037, 65.7037037, 72.7037037, 59.7037037,
       49.7037037, 61.7037037, 68.7037037, 69.7037037])

In [132]:
import numpy
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

def standard_deviation(equation_inputs, pop):
    std = []
    for s in range(len(pop)):
        mean = sum(pop[:,0][s]) / len(pop)
        print(mean)
        var = ((sum(pop[:,0][s]) - mean)**2) / len(equation_inputs)
        print(var)
        std.append(var ** 0.5)
    return std

def calculate_distance(order, dist_x, dist_y):
    distance = ((dist_x-order[1])**2 + (dist_y-order[2])**2 + (order[1]-order[4])**2 + (order[2]-order[5])**2) ** 0.5
    distance = numpy.round(distance, 2)
    distance = distance.reshape(-1,1)
    return distance
    # return scaler.fit_transform(distance)

def calculate_time(order, dist_x, dist_y, speed, pop):
    distance = calculate_distance(order, dist_x, dist_y)
    time = []
    for p in range(len(equation_inputs)):
        t = distance[p] / new_population[:,p]
        time.append(t)
    return time



def calculate_trust():
    return #성공여부도 확률값 95% 0~1 randoms 0.95 작으면 성공

# fitness = ga.cal_pop_fitness(equation_inputs, new_population)
# def cal_pop_fitness(delivered_num, order, dist_x, dist_y, speed, pop):
def cal_pop_fitness(delivered_num, order, dist_x, dist_y, pop):
    std = standard_deviation(delivered_num,pop)
    dist = calculate_distance(order, dist_x, dist_y)
    # time = calculate_time(order, dist_x, dist_y, speed)
    # trust = calculate_trust()
# distance = calculate_distance(order, pop)
# time = calculate_time(order,pop)
    # Calculating the fitness value of each solution in the current population.
    # The fitness function calulates the sum of products between each input and its corresponding weight.
    fitness = 1/std + dist #/time #+ trust
    # fitness = numpy.sum(pop*equation_inputs, axis=1)
    return fitness